In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from scipy.spatial.distance import cdist
from sklearn import metrics
torch.manual_seed(1)

# Dataset

In [ ]:
import string
df = pd.read_csv('/content/drive/My Drive/Project/dataset2.csv').dropna()
total_words = 0
for i in range(len(df['type'])):
  try:
    line = df['post'][i]
    line = line.translate(str.maketrans('', '', string.punctuation))
    total_words += len(line.split())
    if i < 5:
      print(line)
      print(total_words)
  except:
    continue
print(total_words)
print(total_words/len(df['type']))

What has been the most lifechanging experience in your life
10
May the PerC Experience immerse you
16
Hello ENFJ7 Sorry to hear of your distress Its only natural for a relationship to not be perfection all the time in every moment of existence Try to figure the hard times as times of growth as
53
Welcome and stuff
56
Prozac wellbrutin at least thirty minutes of moving your legs and I dont mean moving them while sitting in your same desk chair weed in moderation maybe try edibles as a healthier alternative
89
10739789
27.08490806334059


### Turn balanced datasets to pickle for easy access in the future

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Project/balanced_EI_dataset.csv')
for i in range(len(df['type'])):
  if "E" in df['type'][i]:
    df['type'][i] = 1
  else:
    df['type'][i] = 0

In [ ]:
df.to_pickle("/content/drive/My Drive/Project/EIbinary.pkl")  

In [3]:
df = pd.read_pickle("/content/drive/My Drive/Project/EIbinary.pkl") 

# Lemmatize

In [8]:
import nltk
import string

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    # line = text.translate(str.maketrans('', '', string.punctuation))
    line = [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
    return ' '.join(map(str,line))

df['post'] = df.post.apply(lemmatize_text)
# df['post'] = df['post'].str.lower()

# Traditional Models

Split dataset to training and testing dataset

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['post'], df['type'], 
                                                    test_size=0.2,
                                                    random_state=123)

In [50]:
y_train = y_train.astype('int')
y_test = y_test.astype('int')

Use tf-idf weighted document-term matrix as features

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english')
X_train = tfidf.fit_transform(X_train)
X_test = tfidf.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression

model1 = LogisticRegression(max_iter=2000)
model1.fit(X_train, y_train)
model1.score(X_test, y_test)

0.6214937850621494

In [ ]:
from sklearn.linear_model import SGDClassifier

model2 = SGDClassifier()
model2.fit(X_train, y_train)
model2.score(X_test, y_test)

0.6103013969860301

In [ ]:
from sklearn.linear_model import Perceptron

model3 = Perceptron()
model3.fit(X_train, y_train)
model3.score(X_test, y_test)

0.5620668793312067

In [ ]:
from sklearn.naive_bayes import MultinomialNB

model4 = MultinomialNB()
model4.fit(X_train, y_train)
model4.score(X_test, y_test)

0.6160350896491035

In [ ]:
from sklearn.svm import LinearSVC

model5 = LinearSVC()
model5.fit(X_train, y_train)
model5.score(X_test, y_test)

0.6115388846111539

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import accuracy_score  

model6 = KNeighborsRegressor(n_neighbors=3)
model6.fit(X_train, y_train)
prediction = model6.predict(X_test).astype('int')
accuracy = accuracy_score(y_test, prediction)
print(accuracy)


0.5012787372126278
